# Panda3D Hello World

![](https://www.panda3d.org/wp-content/uploads/2018/12/panda3d_logo_s_white.png)

_Mikołaj Leszczuk_

Na podstawie: [A Panda3D Hello World Tutorial](https://docs.panda3d.org/1.10/python/introduction/tutorial/index)

Zrobimy typowy przykład prostego programu Panda3D. Przejście przez ten moduł kursu pozwoli Wam uzyskać ograniczoną znajomość API Panda3D bez konieczności uczenia się wszystkiego. Program, który stworzymy, załaduje małą scenkę zawierającą trawę i pandę. Panda będzie animowana, aby chodzić tam iz powrotem po trawie.

## Uruchamianie Panda3D

### Tworzenie nowej aplikacji Panda3D

#### ShowBase

Aby uruchomić Panda3D, utwórzcie plik tekstowy i zapiszcie go z rozszerzeniem .py. Użyjcie PyCharm lub innego edytora tekstu specyficznego dla Pythona. Wpiszcie następujący kod źródłowy Pythona:

```python
from direct.showbase.ShowBase import ShowBase


class MyApp(ShowBase):

    def __init__(self):
        ShowBase.__init__(self)


app = MyApp()
app.run()
```

Tutaj uczyniliśmy naszą główną klasę dziedziczeniem z `ShowBase`. Ta klasa ładuje większość innych modułów Panda3D i powoduje pojawienie się okna 3D. Metoda `run()` zawiera główną pętlę Panda3D. Renderuje klatkę, obsługuje zadania w tle, a następnie powtarza. Normalnie niczego nie zwraca, więc musi być wywołana tylko raz i musi być ostatnią linią w skrypcie. W tym konkretnym przykładzie nie będzie nic do renderowania, więc powinniście spodziewać się okna zawierającego pusty szary obszar.

### Uruchamianie programu

Jeśli Panda3D została poprawnie zainstalowana, pojawi się szare okno zatytułowane _Panda_. Nic nie możemy zrobić z tym oknem, ale to się wkrótce zmieni.

## Ładowanie trawiastej scenerii

### Scene Graph

Panda3D zawiera strukturę danych o nazwie _Scene Graph_ (pol. _wykres sceny_, _graf sceny_). Wykres sceny to drzewo zawierające wszystkie obiekty, które mają być renderowane. U podstawy drzewa znajduje się obiekt o nazwie _render_. Nic nie jest renderowane, dopóki nie zostanie po raz pierwszy wstawione do grafu sceny.

Aby zainstalować model trawiastej scenerii w Scene Graph, używamy metody `reparentTo()`. Ustawia to rodzica modelu, tym samym dając mu miejsce na wykresie sceny. W ten sposób model będzie widoczny na scenie.

Na koniec dostosowujemy pozycję i skalę modelu. W tym konkretnym przypadku model środowiska jest trochę za duży i nieco przesunięty dla naszych celów. Procedury (metody) `setScale()` i `setPos()` zmieniają skalę i wyśrodkują model.

Panda3D używa „geograficznego” układu współrzędnych, gdzie pozycja (-8, 42, 0) oznacza współrzędne mapy (8, 42) i wysokość 0. Jeśli jesteście przyzwyczajeni do współrzędnych OpenGL/Direct3D, trzymajcie prawą rękę w klasycznej pozycji z kciukiem jako X, palcami jako Y, a dłonią jako Z skierowanymi do siebie; następnie przechylcie do tyłu, aż Wasza ręka znajdzie się na poziomie z palcami skierowanymi na zewnątrz i dłonią skierowaną do góry. Poruszanie się „do przodu” w Panda3D to pozytywna zmiana we współrzędnej Y.

### Program

#### Zaktualizujcie kod

Przy poprawnym działaniu Panda3D można teraz załadować trawiastą scenerię. Zaktualizujmy swój kod w następujący sposób:

```python
from direct.showbase.ShowBase import ShowBase


class MyApp(ShowBase):

    def __init__(self):
        ShowBase.__init__(self)

        # Załaduj model środowiska.
        self.scene = \
            self.loader.loadModel("models/environment")
        # Zmień model do renderowania.
        self.scene.reparentTo(self.render)
        # Zastosuj przekształcenia skali i pozycji w modelu.
        self.scene.setScale(0.25, 0.25, 0.25)
        self.scene.setPos(-8, 42, 0)


app = MyApp()
app.run()
```

Procedura ShowBase `loader.loadModel()` ładuje określony plik, w tym przypadku plik `environment.egg` w folderze modeli. Zwracana wartość jest obiektem klasy `NodePath`, czyli faktycznie wskaźnikiem do modelu. Zauważcie, że Panda Filename Syntax używa ukośnika, nawet w systemie Windows.

#### Uruchomcie program

Uruchomcie program. Powinniście to zobaczyć:

![](https://docs.panda3d.org/1.10/_images/tutorial1.jpg)

Skała i drzewo wydają się unosić w powietrzu. Kamera jest nieco pod ziemią, a usuwanie tylnej części ścianek sprawia, że ziemia jest dla nas niewidoczna. Jeśli zmienimy położenie kamery, teren będzie wyglądał lepiej.

## Sterowanie kamerą

### Domyślny system sterowania kamerą

Domyślnie Panda3D uruchamia zadanie, które pozwala poruszać kamerą za pomocą myszy.

Klawisze do nawigacji to:

|**Przycisk myszy**|**Akcja**|
|-|-|
|Lewy przycisk|Przesuwanie się w lewo i w prawo.|
|Prawy przycisk|Poruszanie się do przodu i do tyłu.|
|Środkowy przycisk|Obracanie się wokół punktu z początku uruchomienia aplikacji.|
|Prawy i środkowy przycisk|Obracanie punktu widzenia wokół osi widoku.|

Wypróbujcie ten system sterowania kamerą. Problem polega na tym, że czasami jest to niezręczne. Nie zawsze łatwo jest skierować kamerę w wybranym przez nas kierunku.

### Zadania

#### Zaktualizujcie kod

Zamiast tego napiszemy zadanie, które wyraźnie kontroluje pozycję kamery. Zadanie to nic innego jak procedura, która jest wywoływana w każdej klatce. Zaktualizujcie swój kod w następujący sposób:

```python
from math import pi, sin, cos

from direct.showbase.ShowBase import ShowBase

from direct.task import Task


class MyApp(ShowBase):
    def __init__(self):
        ShowBase.__init__(self)

        # Załaduj model środowiska.
        self.scene = \
            self.loader.loadModel("models/environment")
        # Zmień model do renderowania.
        self.scene.reparentTo(self.render)
        # Zastosuj przekształcenia skali i pozycji w modelu.
        self.scene.setScale(0.25, 0.25, 0.25)
        self.scene.setPos(-8, 42, 0)

        # Dodaj procedurę spinCameraTask do menedżera zadań.
        self.taskMgr.add(
            self.spinCameraTask, 
            "SpinCameraTask"
        )

    # Zdefiniuj procedurę poruszania kamerą.
    def spinCameraTask(self, task):
        angleDegrees = task.time * 6.0
        angleRadians = angleDegrees * (pi / 180.0)
        self.camera.setPos(
            20 * sin(angleRadians), 
            -20 * cos(angleRadians), 
            3
        )
        self.camera.setHpr(angleDegrees, 0, 0)
        return Task.cont


app = MyApp()
app.run()
```

Procedura `taskMgr.add()` mówi menedżerowi zadań Panda3D, aby wywoływał procedurę `spinCameraTask()` w każdej klatce. Jest to procedura, którą napisaliśmy, aby sterować kamerą. Dopóki procedura `spinCameraTask()` zwróci stałą `AsyncTask.DS_cont`, menedżer zadań będzie wywoływał ją w każdej klatce.

W naszym kodzie procedura `spinCameraTask()` oblicza żądaną pozycję kamery na podstawie czasu, jaki upłynął. Kamera obraca się o 6 stopni co sekundę. Pierwsze dwie linie obliczają pożądaną orientację kamery; najpierw w stopniach, a potem w radianach. Wywołanie `setPos()` faktycznie ustawia pozycję kamery. (Pamiętajcie, że Y jest poziome, a Z pionowe, więc położenie jest zmieniane przez animację X i Y, podczas gdy Z pozostaje ustawione na 3 jednostki nad poziomem gruntu.) Wywołanie `setHpr()` faktycznie ustawia orientację.

#### Uruchomcie program

Kamera nie powinna już znajdować się pod ziemią; a ponadto powinna teraz obracać się wokół polany:

![](https://docs.panda3d.org/1.10/_images/tutorial2.jpg)

## Ładowanie i animowanie modelu pandy

### Aktorzy

Klasa `Actor` jest przeznaczona dla animowanych modeli. Zauważcie, że używamy `loadModel()` dla modeli statycznych i `Actor` tylko wtedy, gdy są animowane. Dwa argumenty konstruktora dla klasy `Actor` to nazwa pliku zawierającego model i słownik Pythona zawierający nazwy plików zawierających animacje.

### Zaktualizujcie kod

```python
from math import pi, sin, cos

from direct.showbase.ShowBase import ShowBase
from direct.task import Task

from direct.actor.Actor import Actor


class MyApp(ShowBase):
    def __init__(self):
        ShowBase.__init__(self)

        # Załaduj model środowiska.
        self.scene = \
            self.loader.loadModel("models/environment")
        # Zmień model do renderowania.
        self.scene.reparentTo(self.render)
        # Zastosuj przekształcenia skali i pozycji w modelu.
        self.scene.setScale(0.25, 0.25, 0.25)
        self.scene.setPos(-8, 42, 0)

        # Dodaj procedurę spinCameraTask do menedżera zadań.
        self.taskMgr.add(
            self.spinCameraTask,
            "SpinCameraTask"
        )

        # Załaduj i przekształć aktora pandy.
        self.pandaActor = Actor(
            "models/panda-model",
            {"walk": "models/panda-walk4"}
        )
        self.pandaActor.setScale(0.005, 0.005, 0.005)
        self.pandaActor.reparentTo(self.render)
        # Zapętl jego animację.
        self.pandaActor.loop("walk")

    # Zdefiniuj procedurę poruszania kamerą.
    def spinCameraTask(self, task):
        angleDegrees = task.time * 6.0
        angleRadians = angleDegrees * (pi / 180.0)
        self.camera.setPos(
            20 * sin(angleRadians),
            -20 * cos(angleRadians),
            3
        )
        self.camera.setHpr(angleDegrees, 0, 0)
        return Task.cont


app = MyApp()
app.run()
```

Polecenie (metoda) `loop("walk")` powoduje, że animacja spaceru zaczyna się zapętlać.

### Uruchomcie program

Rezultatem jest panda chodząca w miejscu jak na bieżni:

![](https://docs.panda3d.org/1.10/_images/tutorial3.jpg)

## Używanie interwałów do poruszania pandą

### Interwały i sekwencje

#### Interwały

_Interwały_ (ang. _Intervals_) to zadania, które zmieniają właściwość z jednej wartości na drugą w określonym czasie. Rozpoczęcie interwału skutecznie uruchamia proces w tle, który modyfikuje właściwość w określonym przedziale czasu.

#### Sekwencje

_Sekwencje_ (ang. _Sequences_), czasami nazywane _MetaIntervals_, to rodzaj interwału, który zawiera inne interwały. Odtworzenie sekwencji spowoduje, że każdy zawarty interwał będzie wykonywany po kolei, jeden po drugim.

### Program

#### Zaktualizujcie kod

Następnym krokiem jest sprawienie, by panda faktycznie poruszała się tam iz powrotem. Zaktualizuj kod do następującego:

```python
from math import pi, sin, cos

from direct.showbase.ShowBase import ShowBase
from direct.task import Task
from direct.actor.Actor import Actor

from direct.interval.IntervalGlobal import Sequence
from panda3d.core import Point3


class MyApp(ShowBase):
    def __init__(self):
        ShowBase.__init__(self)

        # Wyłącz sterowanie kamery myszką.
        self.disableMouse()

        # Załaduj model środowiska.
        self.scene = \
            self.loader.loadModel("models/environment")
        # Zmień model do renderowania.
        self.scene.reparentTo(self.render)
        # Zastosuj przekształcenia skali i pozycji w modelu.
        self.scene.setScale(0.25, 0.25, 0.25)
        self.scene.setPos(-8, 42, 0)

        # Dodaj procedurę spinCameraTask do menedżera zadań.
        self.taskMgr.add(
            self.spinCameraTask,
            "SpinCameraTask"
        )

        # Załaduj i przekształć aktora pandy.
        self.pandaActor = Actor(
            "models/panda-model",
            {"walk": "models/panda-walk4"}
        )
        self.pandaActor.setScale(0.005, 0.005, 0.005)
        self.pandaActor.reparentTo(self.render)
        # Zapętl jego animację.
        self.pandaActor.loop("walk")

        # Stwórz cztery interwały dla lerpów potrzebne pandzie
        # do chodzenia tam i z powrotem.
        posInterval1 = self.pandaActor.posInterval(
            13,
            Point3(0, -10, 0),
            startPos=Point3(0, 10, 0)
        )
        posInterval2 = self.pandaActor.posInterval(
            13,
            Point3(0, 10, 0),
            startPos=Point3(0, -10, 0)
        )
        hprInterval1 = self.pandaActor.hprInterval(
            3,
            Point3(180, 0, 0),
            startHpr=Point3(0, 0, 0)
        )
        hprInterval2 = self.pandaActor.hprInterval(
            3,
            Point3(0, 0, 0),
            startHpr=Point3(180, 0, 0)
        )

        # Utwórz i odtwórz sekwencję, która koordynuje
        # interwały.
        self.pandaPace = Sequence(posInterval1, hprInterval1,
                                  posInterval2, hprInterval2,
                                  name="pandaPace")
        self.pandaPace.loop()

    # Zdefiniuj procedurę poruszania kamerą.
    def spinCameraTask(self, task):
        angleDegrees = task.time * 6.0
        angleRadians = angleDegrees * (pi / 180.0)
        self.camera.setPos(
            20 * sin(angleRadians),
            -20 * cos(angleRadians),
            3
        )
        self.camera.setHpr(angleDegrees, 0, 0)
        return Task.cont


app = MyApp()
app.run()
```

Po uruchomieniu interwału `pandaPosInterval1` będzie on stopniowo dostosowywał pozycję pandy od (0, 10, 0) do (0, -10, 0) w ciągu 13 sekund. Podobnie po rozpoczęciu interwału `pandaHprInterval1` kurs pandy obróci się o 180 stopni w ciągu 3 sekund.

Sekwencja `pandaPace` powyżej powoduje, że panda porusza się w linii prostej, obraca się, porusza się w przeciwnej linii prostej, a na koniec ponownie się obraca. Kod `pandaPace.loop()` powoduje uruchomienie Sequence w trybie pętli.

### Uruchomcie program

Rezultatem tego wszystkiego jest spowodowanie, że panda będzie chodzić tam iz powrotem od jednego drzewa do drugiego.

## Koniec

Na tym kończy się „Panda Hello World”.